<a href="https://colab.research.google.com/github/da-roth/DiffiPy/blob/main/DifferentiationInterfaceTest/examples-colab/introduction_colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>



In [ ]:
#Install DiffiPy
!pip install diffipy
import diffipy as dp

In [2]:
backend_array = ['numpy', 'torch',  'tensorflow', 'jax']

for backend in backend_array:
    dp.setBackend(backend)
    # Initialize variables and constants
    s0_value = 100.0
    K_value = 110.0
    r_value = 0.05
    sigma_value = 0.3
    dt_value = 1.0
    z_value = 0.5
    N = 1000000
    seed = 1

    # Define variables
    s0 = dp.variable(s0_value, 'input','s0')
    K = dp.variable(K_value, 'input','K')
    r = dp.variable(r_value, 'input','r')
    sigma = dp.variable(sigma_value, 'input','sigma')
    dt = dp.variable(dt_value, 'input','dt')
    z = dp.variable(z_value, 'randomVariableNormal','z')
    input_variables = [s0, K, r, sigma, dt, z]

    # Record Tape: Standard Monte Carlo

    s = s0 * dp.exp((r - sigma **2 / 2) * dt + sigma * dp.sqrt(dt) * z)
    payoff =  dp.if_(s > K, s - K, 0)
    PV_standard = dp.exp(-r * dt) * dp.sum(payoff) / N

    ### Test performance
    dp.seed(seed)
    pre_computed_random_variables = z.NewSample(N) # pre-evaluate random samples

    PV_standard.run_performance_test(input_variables, warmup_iterations = 10, test_iterations = 100)


Backend              Result       Gradient (1. entry)  mean runtime    variance runtime
nump                 10.023817    0.500207             0.036715        0.000028       
numpy_jit            10.024070    0.500215             0.051277        0.000012       
torch                10.020878    0.499224             0.004541        0.000002       
torch_optimized      10.020878    0.499224             0.002874        0.000003       
tensorflow           9.999600     0.499357             0.051064        0.000077       
tensorflow_optimized 9.999600     0.499357             0.039388        0.000056       
jax                  10.033216    0.499721             0.039275        0.006786       
